In [277]:
#!pip install plotguy
#!pip install hkfdb
#!pip install yfinance --upgrade
#!pip install talib-binary

In [278]:
import os
import copy
import datetime
import time
import itertools

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
#import plotguy

import yfinance as yf
import talib as ta

In [279]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Defining Data Folder Structures

In [280]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

Defining Tickers List, Date Range and Data Retrival Intervals

In [288]:
# define list of tickers to backtest
ticker_list = ['ARCC', 'STAG', 'O', 'WPC', 'MAIN']

# define date range
start_date = '2021-03-15'
end_date = '2022-11-10'
interval = '1d'

# initial capital in USD
initial_capital = 100000
last_realized_capital = copy.deepcopy(initial_capital)

**Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [282]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column
# Other indicators: 14 day RSI
for ticker in ticker_list:
  globals()[ticker] = pd.DataFrame
  globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True)
  globals()[ticker]['200EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=200) 
  globals()[ticker]['ATR'] = ta.ATR(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], timeperiod=14)
  globals()[ticker]['RSI'] = ta.RSI(globals()[ticker]['Close'], timeperiod=14)
  globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
  globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)
  globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)
  globals()[ticker] = globals()[ticker].drop('Volume', axis=1)
  globals()[ticker] = globals()[ticker].round(4).dropna()
  # writing each ticker dataframe into separate .csv files, save to google drive
  #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [283]:
df_list = [ARCC, STAG, O, WPC, MAIN]

**Back Testing** </b>


*   Backtesting of Trade Logic across all tickers and in the designated timeframe
*   Backtesting for Trade strategy with simulated PNL



In [284]:
# Class to call Pandas by row and column
# https://stackoverflow.com/questions/28754603/indexing-pandas-data-frames-integer-rows-named-columns 
class XLocIndexer:
    def __init__(self, frame):
        self.frame = frame
    
    def __getitem__(self, key):
        row, col = key
        return self.frame.iloc[row][col]

pd.core.indexing.IndexingMixin.xloc = property(lambda frame: XLocIndexer(frame))

In [285]:
def backtest(df):
  pos_opened = False
  open_price  = 0
  close_price = 0
  pnl = 0
  pnl_list = []
  start_index = 1
  end_index = len(df)


  for i in range(start_index, end_index):
    now_open = df.xloc[i, 'Open']
    now_high = df.xloc[i, 'High']
    now_low  = df.xloc[i, 'Low']
    now_close = df.xloc[i, 'Close']
    now_200EMA = df.xloc[i, '200EMA']
    now_rsi   = df.xloc[i, 'RSI']
    now_MACD        = df.xloc[i, 'MACD']
    now_MACD_signal = df.xloc[i, 'MACD_signal']

    # opening a position
    if (pos_opened == False) and (now_rsi < 30) or ((now_MACD > now_MACD_signal) and (now_MACD >1)) or (now_close < now_200EMA):
      pos_opened = True
      open_price = now_close

    # closing a position
    if (pos_opened == True) and (now_rsi > 65) or ((now_MACD < now_MACD_signal) and (now_MACD <1)):
      pos_opened = False
      close_price = now_close
      pnl = close_price - open_price
      pnl_list.append(pnl)

  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/num_of_trade, 2)

  return num_of_trade, total_profit, avg_pnl

**Calculating Total PNL**

In [286]:
backtest_values = []
for ticker in df_list:
  result = backtest(ticker)
  backtest_values.append(list(result))

backtest_dict = dict(zip(ticker_list, backtest_values))
backtest_raw = pd.DataFrame(backtest_dict.items(), columns=['Ticker', 'Values'])
backtest_result = pd.concat([backtest_raw['Ticker'], pd.DataFrame(backtest_raw['Values'].tolist())],axis = 1)
backtest_result.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade']
Full_PNL = backtest_result['Total PNL'].sum().round(2)

In [287]:
print('Trade period:', start_date, 'to', end_date)
print('Total PNL: USD', Full_PNL)
backtest_result

Trade period: 2021-03-15 to 2022-11-10
Total PNL: USD 2216.53


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade
0,ARCC,95,610.28,6.42
1,STAG,124,-26.82,-0.22
2,O,118,1528.95,12.96
3,WPC,106,20.87,0.20
4,MAIN,122,83.25,0.68
